In [1]:
# Prevent pycache from being generated while developing
import sys
sys.dont_write_bytecode=True

In [2]:
# Import required packages
import ee
from pprint import pprint

In [3]:
# Import local files
import config
import geelayers
import cities
import helpers
import super_rasters

GHSL PROJ: EPSG:3857 [100, 0, -18041000, 0, -100, 9000000]


In [14]:
# Reload local files while developing
import importlib
# importlib.reload(cities)
# importlib.reload(geelayers)
# importlib.reload(config)
# importlib.reload(helpers)
# importlib.reload(super_rasters)

<module 'config' from '/Users/Chris.Rowe/Github/urban_extent/gee/config.py'>

In [5]:
# Create map
#importlib.reload(geelayers)
geelayers.interactive_map()

In [6]:
size = geelayers.GHSL_2020.size().getInfo()
print(f"Image Collection Size: {size}")
print(f'DEST: {config.IC_ID}')
#pprint(geelayers.CITY_DATA.sort('UC_NM_MN').aggregate_array('UC_NM_MN').getInfo())

Image Collection Size: 4241
DEST: data/test_chris_Apr2024/builtup_density_JRCs_50compare_L2_100


In [9]:
geelayers.CITY_DATA = geelayers.CITY_DATA.sort('P15', False)
OFFSET = config.OFFSET
LIMIT = config.LIMIT

if OFFSET and LIMIT:
    LIMIT = LIMIT+OFFSET
IDS = geelayers.CITY_DATA.aggregate_array('ID_HDC_G0').getInfo()[OFFSET:LIMIT]

pprint(len(IDS))

13135


In [10]:
# Map the function over the feature collection
geelayers.CITY_DATA = geelayers.CITY_DATA.map(helpers.replace_geometry)

In [15]:
# Get list of cities
CITIES_LIST = cities.id_hdc_g0_250[1:2]
CITIES_LIST

[9872]

In [16]:
# Calculate Urban Extent for a list of cities
super_rasters.get_urban_extents(IDS, CITIES_LIST)











5 9872


5: Dhaka [9872]
TASK SUBMITTED: Dhaka-9872-2020 {'state': 'READY', 'description': 'Dhaka-9872-2020', 'priority': 100, 'creation_timestamp_ms': 1712893548783, 'update_timestamp_ms': 1712893548783, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'EVRIF4TLIH5ASDAOEFMSVINE', 'name': 'projects/earthengine-legacy/operations/EVRIF4TLIH5ASDAOEFMSVINE'} 















































































































































































































































































































































































































































































































































































































































In [ ]:
# Filter to limit cities to be run to those not already in the output ImageCollection.
COMPLETED_IDS = ee.ImageCollection(config.IC_ID).aggregate_array('ID_HDC_G0')
COMPLETED_FILTER = ee.Filter.And(ee.Filter.inList('ID_HDC_G0', COMPLETED_IDS), ee.Filter.equals('builtup_year', config.mapYear))
COMPLETED_CITIES_LIST = ee.ImageCollection(config.IC_ID).filter(COMPLETED_FILTER).aggregate_array('ID_HDC_G0')

if config.USE_COMPLETED_FILTER:
    geelayers.CITY_DATA = geelayers.CITY_DATA.filter(ee.Filter.inList('ID_HDC_G0', COMPLETED_CITIES_LIST).Not())
else:
    geelayers.CITY_DATA = geelayers.CITY_DATA

pprint(geelayers.CITY_DATA.aggregate_array('UC_NM_MN').getInfo())
pprint(geelayers.CITY_DATA.aggregate_array('GRGN_L2').getInfo())
pprint(geelayers.CITY_DATA.aggregate_array('ID_HDC_G0').getInfo())